In [1]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, classification_report
os.chdir(r"C:\Users\dsala\Downloads")
seed=202013547
# Cargar los datos
train_hogares = pd.read_csv('train_hogares_final.csv')
test_hogares = pd.read_csv('test_hogares_final.csv')


In [2]:
# Definir las características (X) y la variable objetivo (y) del conjunto de entrenamiento
X_train = train_hogares.drop(columns=['id', 'Pobre','Dominio'])  # eliminar variables que no se van a usar
y_train = train_hogares['Pobre']  # 

# Definir las características del conjunto de prueba, eliminando 'id'
X_test = test_hogares.drop(columns=['id','Dominio'])


In [3]:
# Identificar las columnas que están en X_train pero no en X_test
columnas_sobrantes= set(X_train.columns) - set(X_test.columns)

# Eliminar esas columnas de X_train
X_train_reducido = X_train.drop(columns=columnas_sobrantes)

# Alinear el conjunto de prueba con las columnas de X_train
X_test = X_test[X_train_reducido.columns]

# Verificar que los conjuntos tengan las mismas columnas
print(X_train_reducido.columns == X_test.columns)


[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True]


In [9]:
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import pandas as pd

# Entrenar Random Forest con los mejores parámetros
rf_model = RandomForestClassifier(
    n_estimators=200,
    max_depth=20,
    min_samples_split=7,
    min_samples_leaf=5,
    random_state=seed
)

rf_model.fit(X_train_reducido, y_train)

# Entrenar XGBoost con los mejores parámetros
xgb_model = xgb.XGBClassifier(
    gamma=0.1,
    learning_rate=0.1,
    max_depth=7,
    min_child_weight=3,
    n_estimators=200,
    subsample=0.8,
    use_label_encoder=False,
    eval_metric='logloss',
    random_state=seed
)

xgb_model.fit(X_train_reducido, y_train)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=0.1, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.1, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=7,
              max_leaves=None, min_child_weight=3, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=200,
              n_jobs=None, num_parallel_tree=None, random_state=202013547, ...)

In [10]:
# Predicciones en el conjunto de entrenamiento para Random Forest
y_train_pred_rf = rf_model.predict(X_train_reducido)

# Reporte de clasificación para Random Forest
print("Reporte de clasificación para Random Forest en el conjunto de entrenamiento:")
print(classification_report(y_train, y_train_pred_rf))

# Predicciones en el conjunto de entrenamiento para XGBoost
y_train_pred_xgb = xgb_model.predict(X_train_reducido)

# Reporte de clasificación para XGBoost
print("Reporte de clasificación para XGBoost en el conjunto de entrenamiento:")
print(classification_report(y_train, y_train_pred_xgb))


Reporte de clasificación para Random Forest en el conjunto de entrenamiento:
              precision    recall  f1-score   support

           0       0.92      0.98      0.95    131936
           1       0.88      0.64      0.74     33024

    accuracy                           0.91    164960
   macro avg       0.90      0.81      0.85    164960
weighted avg       0.91      0.91      0.91    164960

Reporte de clasificación para XGBoost en el conjunto de entrenamiento:
              precision    recall  f1-score   support

           0       0.91      0.96      0.93    131936
           1       0.77      0.61      0.68     33024

    accuracy                           0.89    164960
   macro avg       0.84      0.78      0.81    164960
weighted avg       0.88      0.89      0.88    164960



In [13]:
# Obtener la importancia de las características de Random Forest
importancia_rf = rf_model.feature_importances_

# Crear un DataFrame para visualizar las importancias en Random Forest
importancia_rf_df = pd.DataFrame({
    'Característica': X_train_reducido.columns,
    'Importancia': importancia_rf
})

# Ordenar de mayor a menor importancia
importancia_rf_df = importancia_rf_df.sort_values(by='Importancia', ascending=False)

# Mostrar las 10 características más importantes
print("Importancia de las características en Random Forest:")
print(importancia_rf_df.head(10))


Importancia de las características en Random Forest:
            Característica  Importancia
23     Proporción_Ocupados     0.122774
5                    P5130     0.107949
27        Num_Dependientes     0.051580
6                    P5140     0.051561
11                   Fex_c     0.044111
24  Promedio_Edad_Ocupados     0.044084
8                  Npersug     0.043265
13                Fex_dpto     0.043050
10                      Lp     0.040892
9                       Li     0.039763


In [12]:
# Obtener la importancia de las características de XGBoost
importancia_xgb = xgb_model.feature_importances_

# Crear un DataFrame para visualizar las importancias en XGBoost
importancia_xgb_df = pd.DataFrame({
    'Característica': X_train_reducido.columns,
    'Importancia': importancia_xgb
})

# Ordenar de mayor a menor importancia
importancia_xgb_df = importancia_xgb_df.sort_values(by='Importancia', ascending=False)

# Mostrar las 10 características más importantes
print("Importancia de las características en XGBoost:")
print(importancia_xgb_df.head(10))


Importancia de las características en XGBoost:
                    Característica  Importancia
37                    Max_Superior     0.300874
23             Proporción_Ocupados     0.151849
8                          Npersug     0.062462
5                            P5130     0.062427
21  Indicador_Hacinamiento_Critico     0.057037
0                            Clase     0.051526
18    Vivienda_Posesion_Sin_Titulo     0.034625
27                Num_Dependientes     0.028409
6                            P5140     0.021302
38     Num_Personas_Media_Superior     0.020613
